# Querying the Graph to study the universe. 


Eventually this will be an admin page. For now I'm just researching it. 


In [1]:
import sys, os
import pandas as pd
import altair as alt
sys.path.append('..')

import  ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 

nest_asyncio.apply()

PII_cols = ['username','userguid','owner']

Most of the `asyncio` and `ssl` stuff is only used for notebooks, it's just here to demo the module. 

In [2]:
from app.connectors.cmdb_graph import CosmosdbClient

executing local windows deployment


connecting using the same module in the app. 

In [3]:
c = CosmosdbClient()


Testing the co

In [4]:
c.run_query()
c.res

[8002]

## Looking at a particular planet

In [7]:
location_id = "4162230164127"
planet_query = f"g.V().has('objid', '{location_id}').valuemap()"
print(planet_query)
c.run_query(planet_query)
c.clean_nodes(c.res)

g.V().has('objid', '4162230164127').valuemap()


[{'name': 'Leyeastpan',
  'class': 'terrestrial',
  'objid': '4162230164127',
  'pop_cap': 100,
  'atmosphere': '[{Argon: 0.185}, {Carbon Dioxide: 0.0}, {Helium: 0.037}, {Hydrogen: 0.231}, {Methane: 0.004}, {Nitrogen: 0.353}, {Oxygen: 0.104}, {Sodium: 0.087}]',
  'radius': 0.697,
  'mass': 0,
  'orbitsDistance': 0.682,
  'orbitsId': '7405736440710',
  'orbitsName': 'Nakly',
  'isSupportsLife': 'true',
  'isPopulated': 'true',
  'isHomeworld': 'true',
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'planet',
  'id': '4162230164127'}]

In [8]:
meta_count = f"g.V().has('objid', '{location_id}').out().groupcount().by('label')"


c.run_query(meta_count)
c.res

[{'star': 1, 'system': 1, 'resource': 4}]

In [9]:
meta_count = f"g.V().has('objid', '{location_id}').in().groupcount().by('label')"


c.run_query(meta_count)
c.res

[{'moon': 2, 'pop': 7, 'biome': 1}]

In [10]:
meta_count = f"g.V().has('objid', '{location_id}').out('has').valuemap()"
c.run_query(meta_count)
c.res

[{'volume': [1223.0],
  'name': ['organics'],
  'objid': ['3771548359034'],
  'max_volume': [1223],
  'description': ['bilogical material that can be consumed by pops'],
  'replenish_rate': [10],
  'userguid': ['8d5b667f-b225-4641-b499-73b77558ff86'],
  'objtype': ['resource']},
 {'name': ['common minerals'],
  'objid': ['2808748861681'],
  'volume': [86],
  'max_volume': [86],
  'description': ['Iron and other common material used in constructuion of infrastructure'],
  'userguid': ['8d5b667f-b225-4641-b499-73b77558ff86'],
  'objtype': ['resource']},
 {'name': ['rare minerals'],
  'objid': ['1000531404900'],
  'volume': [61],
  'max_volume': [61],
  'description': ['lithium, silver and other rare minerals used in infrastructure and technology'],
  'userguid': ['8d5b667f-b225-4641-b499-73b77558ff86'],
  'objtype': ['resource']},
 {'name': ['water'],
  'objid': ['2834729569799'],
  'volume': [9594],
  'max_volume': [9594],
  'description': ['H2O ready to be consumed, either frozen or in

Ok. So why aren't they starving. 

In [14]:
meta_count = f"g.V().has('label', 'action').valuemap()"
c.run_query(meta_count)
pd.DataFrame(c.clean_nodes(c.res))

,type,comment,effort,applies_to,owned_by,building,created_at,to_build,objid,userguid,objtype,id,requires_attr,augments_self_properties,description,name
0,construction,constructing a Department of Forestry,10,pop,pop,forestry,55183,"{type: forestry, label: building, name: Depart...",7166600645413,8d5b667f-b225-4641-b499-73b77558ff86,action,7166600645413,NaN,NaN,NaN,NaN
1,build_infrastructure,NaN,3,pop,NaN,NaN,55190,NaN,4258425703967,8d5b667f-b225-4641-b499-73b77558ff86,action,4258425703967,{wealth: 0.5},"{wealth: -0.1, industry: 0.1}",increase industry by investing in local infras...,build infrastructure
2,build_infrastructure,NaN,3,pop,NaN,NaN,55198,NaN,9918343745025,8d5b667f-b225-4641-b499-73b77558ff86,action,9918343745025,{wealth: 0.1},"{wealth: -0.1, industry: 0.1}",increase industry by investing in local infras...,build infrastructure
3,build_infrastructure,NaN,3,pop,NaN,NaN,55198,NaN,1140951323452,8d5b667f-b225-4641-b499-73b77558ff86,action,1140951323452,{wealth: 0.1},"{wealth: -0.1, industry: 0.1}",increase industry by investing in local infras...,build infrastructure
4,build_infrastructure,NaN,3,pop,NaN,NaN,55198,NaN,2913316203819,8d5b667f-b225-4641-b499-73b77558ff86,action,2913316203819,{wealth: 0.1},"{wealth: -0.1, industry: 0.1}",increase industry by investing in local infras...,build infrastructure


In [16]:
starving_pop_query =f"""
    g.V().has('objid','{location_id}').in('inhabits').valuemap()
    """

c.run_query(starving_pop_query)
starving_pops = c.clean_nodes(c.res)
pd.DataFrame(starving_pops)

,isIdle,name,objid,conformity,literacy,aggression,constitution,health,isIn,industry,wealth,factionLoyalty,userguid,objtype,id
0,true,Dalram Leymon,2775406017619,0.802,0.798,0.315,0.190,0.7,6953342521081,0.2525,0.5252,0.812,8d5b667f-b225-4641-b499-73b77558ff86,pop,2775406017619
1,true,Dalram Dongfieldhal,7646745695043,0.752,0.761,0.268,0.272,0.7,6953342521081,0.3700,0.4155,0.729,8d5b667f-b225-4641-b499-73b77558ff86,pop,7646745695043
2,true,Dalram Sar,1561022813030,0.744,0.667,0.264,0.265,0.7,6953342521081,0.2645,0.4658,0.667,8d5b667f-b225-4641-b499-73b77558ff86,pop,1561022813030
3,true,Dalram Sildrotel,4084988156125,0.708,0.712,0.307,0.214,0.7,6953342521081,0.3605,0.3862,0.677,8d5b667f-b225-4641-b499-73b77558ff86,pop,4084988156125
4,true,Dalram Che,3159775761251,0.800,0.736,0.310,0.119,0.7,6953342521081,0.3145,0.3752,0.782,8d5b667f-b225-4641-b499-73b77558ff86,pop,3159775761251
5,true,Dalram Lu,4377972941053,0.820,0.657,0.277,0.152,0.7,6953342521081,0.3145,0.3358,0.849,8d5b667f-b225-4641-b499-73b77558ff86,pop,4377972941053
6,true,Dalram Fe,6359970696314,0.773,0.654,0.316,0.201,0.7,6953342521081,0.2585,0.4562,0.792,8d5b667f-b225-4641-b499-73b77558ff86,pop,6359970696314
